In [15]:
import numpy as np
import pandas as pd
import datetime
import warnings
import geopy
import pickle

In [55]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="hdb_requests")

In [56]:
with open('../src/data/keys/authkeys', 'rb') as authkeys:
    GOOGLE_AUTH = pickle.load(authkeys)
geolocator = geopy.geocoders.GoogleV3(api_key=GOOGLE_AUTH)

In [57]:
hdbaddress=pd.read_csv('../data/interim/hdbaddresses.csv')
hdbaddress

,0
0,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 172 ..."
1,"Ang Mo Kio, Ang Mo Kio Ave 8 Block Number 510 ..."
2,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 610 ..."
3,"Ang Mo Kio, Ang Mo Kio Ave 10 Block Number 474..."
4,"Ang Mo Kio, Ang Mo Kio Ave 5 Block Number 604 ..."
...,...
9123,"Punggol, Edgedale Plains Block Number 122A Sin..."
9124,"Punggol, Punggol Dr Block Number 677D Singapore"
9125,"Sengkang, Compassvale Dr Block Number 216B Sin..."
9126,"Toa Payoh, Joo Seng Rd Block Number 9 Singapore"


In [58]:
hdbaddress.rename(columns={'0':'address'},inplace=True)

In [59]:
# hdbaddress['loc'] = hdbaddress['address'].apply(geolocator.geocode)

In [60]:
with pd.option_context('display.max_colwidth', None):
  display(hdbaddress)

,address,loc
0,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 172 Singapore","(172 Ang Mo Kio Ave 4, Singapore 560172, (1.3747742, 103.8364795))"
1,"Ang Mo Kio, Ang Mo Kio Ave 8 Block Number 510 Singapore","(510 Ang Mo Kio Ave 8, Singapore 560510, (1.373372, 103.848988))"
2,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 610 Singapore","(610 Ang Mo Kio Ave 4, Singapore 560610, (1.3793747, 103.838986))"
3,"Ang Mo Kio, Ang Mo Kio Ave 10 Block Number 474 Singapore","(474 Ang Mo Kio Ave 10, Singapore 560474, (1.362884, 103.8575989))"
4,"Ang Mo Kio, Ang Mo Kio Ave 5 Block Number 604 Singapore","(604 Ang Mo Kio Ave 5, Singapore 560604, (1.379907, 103.835824))"
...,...,...
9123,"Punggol, Edgedale Plains Block Number 122A Singapore","(Edgedale Plains, Singapore, (1.3984254, 103.9132309))"
9124,"Punggol, Punggol Dr Block Number 677D Singapore","(677D Punggol Dr, Singapore 823678, (1.4061719, 103.9092885))"
9125,"Sengkang, Compassvale Dr Block Number 216B Singapore","(Compassvale Dr, Singapore, (1.3878883, 103.8969344))"
9126,"Toa Payoh, Joo Seng Rd Block Number 9 Singapore","(Joo Seng Rd, Singapore, (1.3363406, 103.8819507))"


In [62]:
hdbaddress['point']= hdbaddress['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [63]:
hdbaddress[['lat', 'lon', 'altitude']] = pd.DataFrame(hdbaddress['point'].to_list(), index=hdbaddress.index)

In [64]:
hdbaddress

,address,loc,point,lat,lon,altitude
0,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 172 ...","(172 Ang Mo Kio Ave 4, Singapore 560172, (1.37...","(1.3747742, 103.8364795, 0.0)",1.374774,103.836479,0.0
1,"Ang Mo Kio, Ang Mo Kio Ave 8 Block Number 510 ...","(510 Ang Mo Kio Ave 8, Singapore 560510, (1.37...","(1.373372, 103.848988, 0.0)",1.373372,103.848988,0.0
2,"Ang Mo Kio, Ang Mo Kio Ave 4 Block Number 610 ...","(610 Ang Mo Kio Ave 4, Singapore 560610, (1.37...","(1.3793747, 103.838986, 0.0)",1.379375,103.838986,0.0
3,"Ang Mo Kio, Ang Mo Kio Ave 10 Block Number 474...","(474 Ang Mo Kio Ave 10, Singapore 560474, (1.3...","(1.362884, 103.8575989, 0.0)",1.362884,103.857599,0.0
4,"Ang Mo Kio, Ang Mo Kio Ave 5 Block Number 604 ...","(604 Ang Mo Kio Ave 5, Singapore 560604, (1.37...","(1.379907, 103.835824, 0.0)",1.379907,103.835824,0.0
...,...,...,...,...,...,...
9123,"Punggol, Edgedale Plains Block Number 122A Sin...","(Edgedale Plains, Singapore, (1.3984254, 103.9...","(1.3984254, 103.9132309, 0.0)",1.398425,103.913231,0.0
9124,"Punggol, Punggol Dr Block Number 677D Singapore","(677D Punggol Dr, Singapore 823678, (1.4061719...","(1.4061719, 103.9092885, 0.0)",1.406172,103.909289,0.0
9125,"Sengkang, Compassvale Dr Block Number 216B Sin...","(Compassvale Dr, Singapore, (1.3878883, 103.89...","(1.3878883, 103.8969344, 0.0)",1.387888,103.896934,0.0
9126,"Toa Payoh, Joo Seng Rd Block Number 9 Singapore","(Joo Seng Rd, Singapore, (1.3363406, 103.88195...","(1.3363406, 103.8819507, 0.0)",1.336341,103.881951,0.0


In [65]:
hdbaddress.to_csv('../data/interim/hdbgeocodes', index=False)